In [2]:
%config IPCompleter.greedy=True
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys, re, math, datetime as dt, pandas as pd, numpy as np, time
import logging
import matplotlib.pyplot as plt
from string import Template
from IPython.display import display, HTML

logging.basicConfig(format='%(asctime)s [%(name)s:%(lineno)d:%(funcName)s] [%(levelname)s] %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_colwidth', 5000)
pd.set_option('display.width', 5000)

def display_df(df):
    display(df.head(4))
    print(df.shape)

In [3]:
seasons = pd.read_csv('../f1_data/seasons.csv')

display_df(seasons)

,year,url
0,2009,http://en.wikipedia.org/wiki/2009_Formula_One_season
1,2008,http://en.wikipedia.org/wiki/2008_Formula_One_season
2,2007,http://en.wikipedia.org/wiki/2007_Formula_One_season
3,2006,http://en.wikipedia.org/wiki/2006_Formula_One_season


(74, 2)


In [4]:
seasons.columns
seasons.isna().values.any()
seasons.duplicated().values.any()

False

In [5]:
seasons = seasons.sort_values(by='year', axis=0, ascending=False)
seasons.head(10)

,year,url
73,2023,https://en.wikipedia.org/wiki/2023_Formula_One_World_Championship
72,2022,http://en.wikipedia.org/wiki/2022_Formula_One_World_Championship
71,2021,http://en.wikipedia.org/wiki/2021_Formula_One_World_Championship
70,2020,http://en.wikipedia.org/wiki/2020_Formula_One_World_Championship
69,2019,http://en.wikipedia.org/wiki/2019_Formula_One_World_Championship
68,2018,http://en.wikipedia.org/wiki/2018_Formula_One_World_Championship
67,2017,http://en.wikipedia.org/wiki/2017_Formula_One_season
66,2016,http://en.wikipedia.org/wiki/2016_Formula_One_season
65,2015,http://en.wikipedia.org/wiki/2015_Formula_One_season
64,2014,http://en.wikipedia.org/wiki/2014_Formula_One_season


In [6]:
status = pd.read_csv('../f1_data/status.csv')
status.head(5)

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [7]:
sprint_results = pd.read_csv('../f1_data/sprint_results.csv')
sprint_results.head(5)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId
0,1,1061,830,9,33,2,1,1,1,3,17,25:38.426,1538426,14,1:30.013,1
1,2,1061,1,131,44,1,2,2,2,2,17,+1.430,1539856,17,1:29.937,1
2,3,1061,822,131,77,3,3,3,3,1,17,+7.502,1545928,17,1:29.958,1
3,4,1061,844,6,16,4,4,4,4,0,17,+11.278,1549704,16,1:30.163,1
4,5,1061,846,1,4,6,5,5,5,0,17,+24.111,1562537,16,1:30.566,1


In [8]:
drivers = pd.read_csv('../f1_data/drivers.csv')
drivers.head(5)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [9]:
circuits = pd.read_csv('../f1_data/circuits.csv')
circuits.head(5)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_International_Circuit
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_International_Circuit
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [13]:
races = pd.read_csv('../f1_data/races.csv')
races.head(5)
races.describe()
races.dtypes


raceId          int64
year            int64
round           int64
circuitId       int64
name           object
date           object
time           object
url            object
fp1_date       object
fp1_time       object
fp2_date       object
fp2_time       object
fp3_date       object
fp3_time       object
quali_date     object
quali_time     object
sprint_date    object
sprint_time    object
dtype: object

In [15]:
races.date = pd.to_datetime(races.date)
races.dtypes
races.sort_index(ascending=False).head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
1100,1120,2023,22,24,Abu Dhabi Grand Prix,2023-11-26,13:00:00,https://en.wikipedia.org/wiki/2023_Abu_Dhabi_Grand_Prix,2023-11-24,09:30:00,2023-11-24,13:00:00,2023-11-25,10:30:00,2023-11-25,14:00:00,\N,\N
1099,1119,2023,21,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_Grand_Prix,2023-11-16,04:30:00,2023-11-16,08:00:00,2023-11-17,04:30:00,2023-11-17,08:00:00,\N,\N
1098,1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Paulo_Grand_Prix,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00
1097,1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City_Grand_Prix,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N
1096,1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_States_Grand_Prix,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00


In [16]:
# races.time.map(lambda x: x.strip() if True else 0)
races.time = pd.to_datetime(races.time, format='%H:%M:%S', errors='coerce')

# races[races.time=='06:00:00']
# races.time.isna().values.any()
# races.time.nunique
# races[races.index==835]
races.dtypes

raceId                  int64
year                    int64
round                   int64
circuitId               int64
name                   object
date           datetime64[ns]
time           datetime64[ns]
url                    object
fp1_date               object
fp1_time               object
fp2_date               object
fp2_time               object
fp3_date               object
fp3_time               object
quali_date             object
quali_time             object
sprint_date            object
sprint_time            object
dtype: object

In [17]:
cols = races.columns[8:17:2]
cols
races[cols] = races[cols].apply(pd.to_datetime, format='%Y-%m-%d',errors='coerce')
races.dtypes

raceId                  int64
year                    int64
round                   int64
circuitId               int64
name                   object
date           datetime64[ns]
time           datetime64[ns]
url                    object
fp1_date       datetime64[ns]
fp1_time               object
fp2_date       datetime64[ns]
fp2_time               object
fp3_date       datetime64[ns]
fp3_time               object
quali_date     datetime64[ns]
quali_time             object
sprint_date    datetime64[ns]
sprint_time            object
dtype: object

In [18]:
cols = races.columns[9:18:2]
cols
races[cols] = races[cols].apply(pd.to_datetime, format='%H:%M:%S', errors='coerce')
races.dtypes

raceId                  int64
year                    int64
round                   int64
circuitId               int64
name                   object
date           datetime64[ns]
time           datetime64[ns]
url                    object
fp1_date       datetime64[ns]
fp1_time       datetime64[ns]
fp2_date       datetime64[ns]
fp2_time       datetime64[ns]
fp3_date       datetime64[ns]
fp3_time       datetime64[ns]
quali_date     datetime64[ns]
quali_time     datetime64[ns]
sprint_date    datetime64[ns]
sprint_time    datetime64[ns]
dtype: object

In [12]:
races.sort_index(ascending=False).head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
1100,1120,2023,22,24,Abu Dhabi Grand Prix,2023-11-26,1900-01-01 13:00:00,https://en.wikipedia.org/wiki/2023_Abu_Dhabi_Grand_Prix,2023-11-24,1900-01-01 09:30:00,2023-11-24,1900-01-01 13:00:00,2023-11-25,1900-01-01 10:30:00,2023-11-25,1900-01-01 14:00:00,NaT,NaT
1099,1119,2023,21,80,Las Vegas Grand Prix,2023-11-19,1900-01-01 06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_Grand_Prix,2023-11-16,1900-01-01 04:30:00,2023-11-16,1900-01-01 08:00:00,2023-11-17,1900-01-01 04:30:00,2023-11-17,1900-01-01 08:00:00,NaT,NaT
1098,1118,2023,20,18,São Paulo Grand Prix,2023-11-05,1900-01-01 17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Paulo_Grand_Prix,2023-11-03,1900-01-01 14:30:00,2023-11-04,1900-01-01 14:30:00,NaT,NaT,2023-11-03,1900-01-01 18:00:00,2023-11-04,1900-01-01 18:30:00
1097,1117,2023,19,32,Mexico City Grand Prix,2023-10-29,1900-01-01 20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City_Grand_Prix,2023-10-27,1900-01-01 18:30:00,2023-10-27,1900-01-01 22:00:00,2023-10-28,1900-01-01 17:30:00,2023-10-28,1900-01-01 21:00:00,NaT,NaT
1096,1116,2023,18,69,United States Grand Prix,2023-10-22,1900-01-01 19:00:00,https://en.wikipedia.org/wiki/2023_United_States_Grand_Prix,2023-10-20,1900-01-01 17:30:00,2023-10-21,1900-01-01 18:00:00,NaT,NaT,2023-10-20,1900-01-01 21:00:00,2023-10-21,1900-01-01 22:00:00


In [16]:
x = [print(n) for n in races.columns]
races.isnull().values.any()
races.duplicated().values.any()
races.head()

raceId
year
round
circuitId
name
date
time
url
fp1_date
fp1_time
fp2_date
fp2_time
fp3_date
fp3_time
quali_date
quali_time
sprint_date
sprint_time


,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,1900-01-01 06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,1900-01-01 09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Grand_Prix,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,1900-01-01 07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Grand_Prix,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,1900-01-01 12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Grand_Prix,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,1900-01-01 12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Grand_Prix,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [19]:
races[races.year==2023]
# races = races.groupby(['year'])
races = races.sort_values(by=['year','round'], axis=0, ascending=[False, True])

races.head(5)
races[races.raceId==1098]['fp1_date'].iloc[0]

Timestamp('2023-03-03 00:00:00')